# <span style="color:seagreen">01-Translate_DICs</span>

In this notebook we will guide you through performing DIC translation. Due to the usage of the DIC filter, a shift in the xy dimension compared to the fluorescent images is introduced. In this notebook we we will perform a translation on the DIC images to correct for this shift. This is necessary as the DIC images are used to create the cell masks by Cellpose.

## 1.0 - Load libraries

In [1]:
import numpy as np
from glob import glob
from pathlib import Path
import re
from skimage import io
import matplotlib.pyplot as plt
from bigfish.detection import detect_spots
from bigfish.stack import remove_background_gaussian
from bigfish.stack import get_in_focus_indices
from bigfish.stack import compute_focus
from scipy.signal import savgol_filter
import napari
import seaborn as sns
from scipy.ndimage import shift
import sys
sys.path.append('../')

from src.misc import group_experiments, load_data, find_high_density_patch, find_in_focus_indices, translate

***

## 1.1 - Load fluorescent channels and DIC of a single image

Here we will examine and explore the created data structure. Next, we will load a DIC and the fluorescent image stack of single field of view (fov).

In [2]:
root_dir = '../Data/restructured_data/replicate1'
experiments = group_experiments(root_dir)
experiments_to_process=list(experiments.keys())

In [3]:
# some descriptions of the provided data
print('These are the strain + mRNAs + condition combinations found:')
experiments_to_process

These are the strain + mRNAs + condition combinations found:


[]

In [ ]:
print('These fovs (field of view, i.e. a single imaging location) are found within a single of these combinations:')
experiments['cbk1_CLB2Q670HWP1CAL610_SPIDER37']

In [ ]:
print('These are the channels and DIC images found within a single fov:')
experiments['cbk1_CLB2Q670HWP1CAL610_SPIDER37'][3]

In [ ]:
# select a single channel of a field of view (fov), i.e a single imaging xy postion with the corresponding DIC image.
fov_id=3

fov=io.imread(experiments['cbk1_CLB2Q670HWP1CAL610_SPIDER37'][fov_id]['CY5'])
DIC=io.imread(experiments['cbk1_CLB2Q670HWP1CAL610_SPIDER37'][fov_id]['DIC'])

## 1.2 - Zprojection of a single channel of a fov

Next we will perform a maximal intensity projection over the in-focus z-layers of a single channel of the fov to create a 2d representation of this channel. This 2D representation will be used to perform image registration of the DIC image and the fluorescent zstack image.

In [ ]:
# perform maximal intensity projection over the z-dimension
zstart= 40 # the starting in-focus layer
zend= 60 # last in-focus layer
proj_fov=np.amax(fov[zstart : zend,...],axis=0)

#plot images
fig, ax= plt.subplots(ncols=2,figsize=(10,10))

#left column plot
ax[0].imshow(proj_fov)
ax[0].set_title('zprojection fluorescent image stack')

#right column plot
ax[1].imshow(DIC)
ax[1].set_title('DIC')

***

## 1.2 - Use the napari viewer + widget to translate the DIC image

Now, we will add the projected fluorescent fov and the DIC image to the napari imaging viewer. Next, a docker widget is added to this viewer. This widget allows you to translate the DIC image using the keys: 'a' (left), 'w' (up), 's' (down), 'd' (right) keys. Note that per key press the DIC image moves only one pixel by default. This can be changed with the stepsize parameter within the widget. The translation coordinates can be printed within this notebook by pressing 't'.

In [ ]:
#instantiate the napar viewer
viewer = napari.Viewer()

viewer.add_image(DIC,  opacity=0.5)
viewer.add_image(proj_fov)


#translation widget for layer alignment
#keybindings are used, w:up,a:left,s:down,d:right. 
#to register and update napari coordinates press t.       
viewer.window.add_dock_widget(translate())

## 1.3 - Batch translation

Execute this code to use the image translation you performed above on all your images. Make sure to paste the image translation coordinates that are printed after **x, y:** above. Note, by modyfying the list in experiments_to_process folder you can only translate a subset of your images.

In [ ]:
root_dir = '../Data/restructured_data/replicate1'
experiments = group_experiments(root_dir)
experiments_to_process=list(experiments.keys())
experiments_to_process=

In [ ]:
root_dir = '../Data/restructured_data/replicate1'

for identifier in experiments_to_process:
    fovs = experiments[identifier]
    
    for fov, paths, in fovs.items():
        print(f'processing {identifier=}, {fov=}')
        data = load_data(paths)
        
        #choose channel to perform spot detection in.
        channel='DIC'
        
        #load channel.
        DIC = data.get(channel)
        
        #copy x,y numbers from print statement above here        
        x,y= -15.0,10.0
        
        #shift x, y
        DIC_shift=shift(DIC,[x,y])
        
        #reformat save location
        DIC_name=Path(paths['DIC']).name
        
        #save DIC, uncomment to save
        io.imsave(Path(root_dir,'Corrected DIC',DIC_name),DIC_shift)